In [84]:
import pandas as pd
import joblib as jb
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score


In [85]:
df = pd.read_csv("dataset.csv")
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
missing_values = df.isnull().sum()
missing_data = pd.DataFrame({
    "Missing Values": missing_values
})
missing_data

,Missing Values
Disease,0
Symptom_1,0
Symptom_2,0
Symptom_3,0
Symptom_4,348
Symptom_5,1206
Symptom_6,1986
Symptom_7,2652
Symptom_8,2976
Symptom_9,3228


In [87]:
# mengubah missing values menjadi "tidak diketahui"
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype == 'object':
            df[col].fillna("tidak diketahui", inplace=True)
        else:
            df[col].fillna(df[col].median(), inplace=True)
df.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_21848\3773304205.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna("tidak diketahui", inplace=True)


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui
3,Fungal infection,itching,skin_rash,dischromic _patches,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui


In [88]:
symptom_columns = ['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 'Symptom_6',
                    'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10', 'Symptom_11', 'Symptom_12',
                    'Symptom_13', 'Symptom_14', 'Symptom_15', 'Symptom_16', 'Symptom_17']
df["Symptoms"] = df[symptom_columns].fillna("").agg(" ".join, axis=1).str.strip()
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17,Symptoms
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,itching skin_rash nodal_skin_eruptions disc...
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,skin_rash nodal_skin_eruptions dischromic _p...
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,itching nodal_skin_eruptions dischromic _pat...
3,Fungal infection,itching,skin_rash,dischromic _patches,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,itching skin_rash dischromic _patches tidak ...
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,tidak diketahui,itching skin_rash nodal_skin_eruptions tidak...


In [89]:

X_train, X_test, y_train, y_test = train_test_split(df["Symptoms"], df["Disease"], test_size=0.2, random_state=42)

model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=False)

accuracy, report


(1.0,
 '                                         precision    recall  f1-score   support\n\n(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        18\n                                   AIDS       1.00      1.00      1.00        30\n                                   Acne       1.00      1.00      1.00        24\n                    Alcoholic hepatitis       1.00      1.00      1.00        25\n                                Allergy       1.00      1.00      1.00        24\n                              Arthritis       1.00      1.00      1.00        23\n                       Bronchial Asthma       1.00      1.00      1.00        33\n                   Cervical spondylosis       1.00      1.00      1.00        23\n                            Chicken pox       1.00      1.00      1.00        21\n                    Chronic cholestasis       1.00      1.00      1.00        15\n                            Common Cold       1.00      1.00      1.00        23\n      

In [90]:
jb.dump(model, "model2.h5")

['model2.h5']

In [91]:
gejala = input("masukan gejala?")
predict = model.predict([gejala])[0]
print(predict)

AIDS
